In [139]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn import CrossEntropyLoss

In [140]:
from sklearn.decomposition import NMF
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
    
from scipy import sparse
from os import path

In [141]:
HOME_DIR = "/recsys_LTX"
ML_DIR = "/Data/ml-1m"

In [142]:
import os
export_dir = os.getcwd()

In [143]:
files_path = export_dir + HOME_DIR + ML_DIR

In [144]:
# Load the data
data = pd.read_csv(files_path+ "/ratings.dat", sep="::", engine="python",
                   names=["user_id_original", "item_id_original", "rating", "timestamp"])

In [145]:
data.describe()

,user_id_original,item_id_original,rating,timestamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


In [146]:
#Load movies data to get genres per movie
movies = pd.read_csv(
    "ml-1m/movies.dat", sep="::", names=["movie_id", "title", "genres"],encoding = "ISO-8859-1"
)

/tmp/ipykernel_95383/3986192099.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv(


In [147]:
#Is used for toy examples to verify the model
genres = [
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]

for genre in genres:
    movies[genre] = movies["genres"].apply(
        lambda values: int(genre in values.split("|"))
    )

In [148]:
movies[movies.movie_id == 1127]

,movie_id,title,genres,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1111,1127,"Abyss, The (1989)",Action|Adventure|Sci-Fi|Thriller,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [149]:
# Convert the ratings to binary values (1 if rating exists, 0 otherwise)
data["rating"] = data["rating"].apply(lambda x: 1 if x > 0 else 0)


# Encode target values
data["user_id"] = LabelEncoder().fit_transform(data.user_id_original)
data["item_id"] = LabelEncoder().fit_transform(data.item_id_original)

# Get the number of users and items in the dataset
num_users = data.user_id.unique().shape[0]
num_items = data.item_id.unique().shape[0]

In [150]:
data[data.item_id == 1050] 

,user_id_original,item_id_original,rating,timestamp,user_id,item_id
329,5,1127,1,978241390,4,1050
1060,10,1127,1,979168108,9,1050
1428,13,1127,1,978202328,12,1050
1863,17,1127,1,978160616,16,1050
2427,19,1127,1,978555732,18,1050
...,...,...,...,...,...,...
997937,6028,1127,1,956723693,6027,1050
998283,6033,1127,1,956713999,6032,1050
998850,6036,1127,1,956716780,6035,1050
999573,6037,1127,1,956718919,6036,1050


In [151]:
user_group = data[["user_id","item_id"]].groupby(data.user_id)

users_data = pd.DataFrame(
    data={
        "user_id": list(user_group.groups.keys()),
        "item_ids": list(user_group.item_id.apply(list)),
    }    
)

In [152]:
users_data

,user_id,item_ids
0,0,"[1104, 639, 853, 3177, 2162, 1107, 1195, 2599,..."
1,1,"[1259, 2853, 1414, 626, 2013, 627, 2078, 2426,..."
2,2,"[3189, 1504, 627, 1295, 3301, 101, 2530, 1120,..."
3,3,"[3235, 1120, 2743, 1124, 971, 253, 1848, 466, ..."
4,4,"[2775, 2140, 1087, 38, 279, 2144, 1413, 1293, ..."
...,...,...
6035,6035,"[557, 560, 1872, 1873, 1877, 574, 575, 3, 2793..."
6036,6036,"[575, 2794, 1305, 1883, 1884, 579, 2802, 842, ..."
6037,6037,"[1316, 859, 2872, 225, 1059, 1066, 1094, 1965,..."
6038,6038,"[574, 1886, 1314, 2809, 2814, 1899, 1902, 1906..."


In [153]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
user_one_hot = pd.DataFrame(mlb.fit_transform(users_data["item_ids"]),columns=mlb.classes_, index=users_data["item_ids"].index)

In [154]:
user_one_hot["user_id"]=users_data["user_id"]

In [155]:
user_one_hot

,0,1,2,3,4,5,6,7,8,9,...,3697,3698,3699,3700,3701,3702,3703,3704,3705,user_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6035
6036,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6036
6037,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6037
6038,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6038


In [156]:
user_one_hot.iloc[2,1327]

1

In [157]:
#Sample items from consumed items to predict (positive examples)
np.random.seed(3)
def get_y_values(numpy_arr):
    print(numpy_arr)
    y_values = []
    users_arr = np.split(numpy_arr[:, 1], np.unique(numpy_arr[:, 0], return_index=True)[1][1:])
    for u in users_arr:
          y_values.append(int(np.random.choice(u[:-2],1,replace=False)))
    return y_values    

In [158]:
y_indices = get_y_values(np.argwhere(user_one_hot.to_numpy()>0))

[[   0    0]
 [   0   47]
 [   0  144]
 ...
 [6039 3508]
 [6039 3575]
 [6039 3706]]


In [159]:
len(y_indices)

6040

In [160]:
user_one_hot["y_positive"]= y_indices

In [161]:
user_one_hot

,0,1,2,3,4,5,6,7,8,9,...,3698,3699,3700,3701,3702,3703,3704,3705,user_id,y_positive
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,708
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1466
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,1169
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,466
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,4,2126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,6035,2839
6036,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6036,1209
6037,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6037,1059
6038,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6038,853


In [162]:
#Create representation of each item as a binary vector
items_values = pd.DataFrame(0, index=np.arange(num_items), columns=np.arange(num_items))

In [163]:
items_values

,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3701,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3702,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3703,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3704,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [164]:
for index, row in items_values.iterrows():
    #items_values.iloc[index,user_one_hot["y_value"][index]] = 1
    items_values.iloc[index][index] = 1

In [165]:
items_values.iloc[581][2681]

0

In [166]:
items_values.index.values

array([   0,    1,    2, ..., 3703, 3704, 3705])

In [167]:
random_state = 12
# The list of users IDs
users_indices = items_values.index.values

# Set the split ratio (80% for training, 20% for testing)
split_ratio = 0.8

# Calculate the split index
shuffled_users_ids = np.random.permutation(users_indices)
split_index = int(len(shuffled_users_ids) * split_ratio)

# Split the list of user IDs into training and testing sets
train_user_ids = shuffled_users_ids[:split_index]
test_user_ids = shuffled_users_ids[split_index:]


In [168]:
train_user_ids

array([3565, 2801, 1964, ..., 1906, 2668, 1879])

In [169]:
# Split the user hot encoding matrix into training and testing sets based on the selected user IDs
train_data = user_one_hot.loc[train_user_ids]
test_data = user_one_hot.loc[test_user_ids]

In [170]:
# Split the item hot encoding matrix into training and testing sets based on the selected user IDs
train_items_values = items_values.loc[train_user_ids]
test_items_values  = items_values.loc[test_user_ids]

In [171]:
train_items_values

,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
3565,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2801,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3048,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1906,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2668,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [172]:
train_data

,0,1,2,3,4,5,6,7,8,9,...,3698,3699,3700,3701,3702,3703,3704,3705,user_id,y_positive
3565,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3565,1164
2801,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2801,2983
1964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1964,2621
3048,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3048,1175
1234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1234,309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1081,2334
2008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2008,2164
1906,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1906,18
2668,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2668,1018


In [173]:
test_data

,0,1,2,3,4,5,6,7,8,9,...,3698,3699,3700,3701,3702,3703,3704,3705,user_id,y_positive
37,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,37,486
2952,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2952,3014
3112,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3112,288
3341,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3341,2201
3461,1,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,3461,1919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,154,2190
751,1,0,1,1,0,1,0,0,0,1,...,0,0,0,1,1,0,0,1,751,1842
1846,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1846,2570
752,1,1,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,752,1993


In [174]:
#Add negative examples to training data
#Calculate the popularity of each item in the training set
popularity_dict = train_data.iloc[:,:-2].sum(axis=1).to_dict()

In [175]:
prob_dict = {}
for k, v in popularity_dict.items():
    prob_dict[k] = v/sum(popularity_dict.values())
#print(prob_dict)

In [176]:
import operator
sorted(prob_dict .items(), key=operator.itemgetter(1))[-1]

(1679, 0.003791916044929081)

In [177]:
#sample negative points 
def get_negative_samples(numpy_arr,num):
    negative_values = []
    users_arr = np.split(numpy_arr[:, 1], np.unique(numpy_arr[:, 0], return_index=True)[1][1:])
    for u in users_arr:
        items_from_dict_keys = [d for d in popularity_dict.keys() if d in u]
        sum_popularity= 0
        for d in items_from_dict_keys:
            sum_popularity += popularity_dict[d] 
        items_probs = [popularity_dict[d]/sum_popularity for d in items_from_dict_keys]
        negative_samples = np.random.choice(items_from_dict_keys,size=num,replace=False,p=items_probs)
        if(num == 1):
            negative_values.append(int(negative_samples))
        else:
            negative_values.append(negative_samples) 
                               
    return negative_values   

In [178]:
y_negative = get_negative_samples(np.argwhere(train_data.iloc[:,:-2].to_numpy()==0), num=1)

In [179]:
train_data["y_negative"] = y_negative

In [180]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,3699,3700,3701,3702,3703,3704,3705,user_id,y_positive,y_negative
3565,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3565,1164,997
2801,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2801,2983,1766
1964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1964,2621,1646
3048,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3048,1175,3356
1234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1234,309,1860


In [181]:
#Positive and Negative examples merged together
train_data_mixed = train_data.merge(train_data.loc[:,['user_id','y_positive','y_negative']].melt('user_id', value_name='y_values').replace({'y_positive': 1, 'y_negative': 0}), on="user_id").rename(columns={'variable': 'interaction'}).drop(['y_positive', 'y_negative'],axis=1)

In [182]:
train_data_mixed.head()

,0,1,2,3,4,5,6,7,8,9,...,3699,3700,3701,3702,3703,3704,3705,user_id,interaction,y_values
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3565,1,1164
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3565,0,997
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2801,1,2983
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2801,0,1766
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1964,1,2621


In [183]:
#len(popularity_dict.keys()) 

2964

In [184]:
#train_items_values[0].describe()

count    2964.000000
mean        0.000337
std         0.018368
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: 0, dtype: float64

In [185]:
#Matrix Factorization Model
#User - Item interaction
#Users are represented as one-hot vector, where 1 is in the index of the item that was consumed by the user
# Items are represented as the one-hot vector where 1 is in the index of the relevant item id
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size, bias = False)
        self.fc2 = nn.Linear(hidden_size, 1, bias = False)
        #self.relu2 = nn.ReLU()
        #self.dropout = nn.Dropout(0.3)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        #x = self.relu2(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x
    
class Recommender(nn.Module):
    def __init__(self, num_users, num_items, hidden_size):
        super(Recommender, self).__init__()
        self.mlp = MLP(num_items*2, hidden_size).to(device)

    def forward(self, user_vector, item_vector):
        concatenated = torch.cat([user_vector, item_vector], dim=-1).to(device)
        output = self.mlp(concatenated.to(torch.float32))
        return output.to(device)

In [ ]:
# Train the model on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"
hidden_dim = 128
print("num_users is ", num_users)
print("num_items is ", num_items)
model = Recommender(num_users,num_items, hidden_dim)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [187]:
train_data_mixed.head()

,0,1,2,3,4,5,6,7,8,9,...,3699,3700,3701,3702,3703,3704,3705,user_id,interaction,y_values
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3565,1,1164
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3565,0,997
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2801,1,2983
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2801,0,1766
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1964,1,2621


In [188]:
train_losses = []
for epoch in range(10):
    for i in range(len(train_data_mixed)):
        item_id = train_data_mixed.y_values.iloc[i]
        user_id = train_data_mixed.user_id.iloc[i]
        item_vector = items_values.iloc[item_id]
        user_vector = train_data_mixed.iloc[i,:-3] - item_vector 
        interact = train_data_mixed.interaction.iloc[i]
        
        user_tensor = torch.LongTensor(user_vector).to(device)
        item_tensor = torch.LongTensor(item_vector).to(device)
        interact_tensor = torch.FloatTensor([interact]).to(device)
        
        optimizer.zero_grad()
        output = (model(user_tensor, item_tensor)).to(device)
        loss = loss_fn(output, interact_tensor)
        
        train_losses.append(loss.item())
            
        loss.backward()
        optimizer.step()

        if i % 10000 == 0:
            print(f"Epoch {epoch}, Step {i}, Loss {loss.item():.4f}")  
            
print("Train BCE: {}".format(np.mean(train_losses)))              

Epoch 0, Step 0, Loss 0.7030
Epoch 1, Step 0, Loss 0.6853
Epoch 2, Step 0, Loss 0.4634
Epoch 3, Step 0, Loss 0.5363
Epoch 4, Step 0, Loss 0.4295
Epoch 5, Step 0, Loss 0.3301
Epoch 6, Step 0, Loss 0.2097
Epoch 7, Step 0, Loss 0.2229
Epoch 8, Step 0, Loss 0.0755
Epoch 9, Step 0, Loss 0.0747
Train BCE: 0.5721681360600682


In [189]:
test_data.head(10)

,0,1,2,3,4,5,6,7,8,9,...,3698,3699,3700,3701,3702,3703,3704,3705,user_id,y_positive
37,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,37,486
2952,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2952,3014
3112,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3112,288
3341,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3341,2201
3461,1,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,3461,1919
452,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,452,699
3001,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3001,2958
118,1,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,118,1564
3495,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,3495,1384
2806,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2806,1765


In [190]:
#Go over items that are not in the user history and calculate probabilities for interaction
def get_top_k(user_vector, num_items, model, top_k):
    item_prob_dict = {}
    for i in range(num_items):
        if(user_vector[i] == 0):
            item_vector = torch.tensor(torch.zeros(num_items)).to(device)
            item_vector[i] = 1
            output_model = model(user_vector, item_vector).to(device)
            item_prob_dict[i]=output_model.item()
    #sort dictionary by probabilities
    sorted_items_by_prob  = dict(sorted(item_prob_dict.items(), key=lambda item: item[1],reverse=True))
    #return k top elements
    return dict(list(sorted_items_by_prob.items())[0:top_k])

In [191]:
#Calculate the index of the user in the ranked list
#For further comparison whether some item removal affects the index or not 
#(we would like to see that it decreases)
def get_index_in_the_list(user_vector, item_id, num_items, model):
    return list(get_top_k(user_vector, num_items, model, num_items).keys()).index(item_id)

In [192]:
#This one is used in the model by default
#Define combined loss function that contains cross entropy between updated user history and item's prediction
#together with the regularization part on the masked vector 
#Find min (two parameters mask and z in order to find  x_masked
# x_masked is returned, where item with the max probability is the one that explains the target item the best
class LossModelCombined(torch.nn.Module):
        def __init__(self,x, y_true,y_pred,alpha_parameter):
            super().__init__()
            self.x = x.to(device)
            self.y_true = y_true.to(device)
            self.y_pred = y_pred.to(device)
            self.mask = torch.nn.Parameter(torch.zeros_like(self.x, dtype=torch.float32, device=device), requires_grad=True) #torch.tensor(self.y_pred), requires_grad=True)    #
            self.z = torch.nn.Parameter(torch.nn.Parameter(torch.zeros_like(self.x, dtype=torch.float32, device=device), requires_grad=True)) #torch.tensor(self.y_pred), requires_grad=True) 
            self.alpha_parameter = alpha_parameter

        def forward(self):
            #x_masked as defined via mask and z: x_m = x o m + z o (1-m)
            x_masked = (self.mask * self.x.to(device) + self.z.to(device)*(torch.ones_like(self.x).to(device)-self.mask)).to(device)
      
            #cross entropy loss between updated vector after removing influencers discovered by x_masked and y_pred that we got from the test item
            cross_entropy_loss = nn.functional.cross_entropy(self.x - x_masked.round(), self.y_pred).to(device)
            
            #regularization function
            l1_loss = torch.mean(torch.abs(x_masked)).to(device)

            loss = cross_entropy_loss + self.alpha_parameter * l1_loss 
            return loss   

In [193]:
# find mask with Gumbel-Softmax distribution
def find_mask_combined(x, y_true, y_pred, alpha_parameter, temperature=0.5, num_samples=10):
    model = LossModelCombined(x, y_true, y_pred, alpha_parameter).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001,capturable=True)

    for epoch in range(1000):
        optimizer.zero_grad()
        loss = model().to(device)
        loss.backward(retain_graph=True)
        optimizer.step()
        
    with torch.no_grad():
        # Sample from the Gumbel-Softmax distribution with temperature
        z = model.z.to(device)
        z = z.view(-1) 
        logits = torch.stack([z, torch.zeros_like(z)], dim=-1) / temperature
        y = torch.distributions.gumbel.Gumbel(0, 1).sample(logits.shape + (num_samples,)).mean(dim=-1)
        
        # Compute the binary mask from the Gumbel-Softmax sample
        mask = nn.functional.softmax(y, dim=-1)[:, 0].view(-1).to(device)
     
        # Calculate x_masked
        x_masked = (mask * model.x.to(device) + z.to(device) * (torch.ones_like(x).to(device) - mask)).to(device)
        
    return x_masked 

In [195]:
# Evaluate the model on the test set
test_losses = []
ndcg_scores = []
y_true = []
y_pred = []
#hr_scores = []
top_k = 20
correct = 0
threshold = 0.5
total = len(test_data)

is_in_top_k = dict.fromkeys(range(num_items),0)
still_in_top_k = dict.fromkeys(range(num_items),0)
items_index = {}

for i in range(len(test_data)):
    item_id = test_data.y_positive.iloc[i]
    user_id = test_data.user_id.iloc[i]
    item_vector = items_values.iloc[item_id]
    user_vector_original = test_data.iloc[i,:-2]
    user_vector = user_vector_original - item_vector 
    interaction = 1

    user_tensor = torch.LongTensor(user_vector).to(device)
    item_tensor = torch.LongTensor(item_vector).to(device)
    interaction_tensor = torch.FloatTensor([interaction]).to(device)
    #user_original_tensor = torch.FloatTensor(user_vector_original).to(device)
    
    output = model(user_tensor,item_tensor).to(device)
    loss = loss_fn(output, interaction_tensor)
    test_losses.append(loss.item())
    
    pred = np.round(output.cpu().detach().numpy()).astype(int)
    #for the calculation of accuracy
    if (pred == interaction):
        correct+=1
    
    #top_k_items = list(get_top_k(user_tensor, num_items, model, top_k).keys())
    #print(top_k_items)
    #print(item_id)
    #if (item_id in top_k_items):
    #    is_in_top_k[item_id] = 1
      
    items_index[item_id] = get_index_in_the_list(user_tensor, item_id, num_items, model)    

    
    pred_tensor = torch.tensor(torch.zeros_like(user_tensor)).to(device)
    pred_tensor[item_id] = output
    masked_vector = find_mask_combined(user_tensor.to(torch.float32), item_tensor.to(torch.float32), pred_tensor.to(torch.float32), alpha_parameter=0.5)
    #print(masked_vector)
    #print(user_tensor.sum())
    max_element = torch.max(masked_vector)
    max_item = torch.argmax(masked_vector)
    #print(item_id,max_item)    
        
    max_item_tensor =  torch.zeros_like(item_tensor).to(device)
    max_item_tensor[max_item] = 1
    updated_tensor = user_tensor - max_item_tensor

    
    
    #print(item_id, get_place_in_the_list(updated_tensor, item_id, num_items, model))
    #decrease/increase in the list of items
    updated_output = model(updated_tensor, item_tensor).to(device)
    
    items_index[item_id] = get_index_in_the_list(updated_tensor, item_id, num_items, model) - items_index[item_id] 
    
    #top_k_items_updated = list(get_top_k(updated_tensor, num_items, model, top_k).keys())
    #print(top_k_items_updated)
    #if (item_id in top_k_items_updated):
    #    still_in_top_k[item_id] = 1
        
    if(i==10):
        break
        
print("Test BCE: {}".format(np.mean(test_losses)))  
print("Test NDCG: {}".format(np.mean(ndcg_scores)))  
print("accuracy " ,correct/total)
#print("sigmoid_decrease ", sigmoid_decrease/total)
#print("sigmoid_decrease_percent ", np.average(sigmoid_decrease_percentage))
#print("sigmoid_increase ", sigmoid_increase/total)
#print("is_in_top_k ",is_in_top_k)
#print("still_in_top_k ",still_in_top_k)
#print("items_index ", items_index)

/tmp/ipykernel_95383/1362357296.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item_vector = torch.tensor(torch.zeros(num_items)).to(device)
/tmp/ipykernel_95383/3958500841.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_tensor = torch.tensor(torch.zeros_like(user_tensor)).to(device)


Test BCE: 0.3879555868175389
Test NDCG: nan
accuracy  0.01078167115902965


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [196]:
#A toy example where user consumed comedy and documentary items and the documentary item was recommended, so we expect that another documentary item will explain this one

#item_id = 96 # Documentary "Heidi Fleiss"
item_id = 3701 # Comedy "Meet the parents"
item_vector = items_values.iloc[item_id]
interaction = 1


item_tensor = torch.LongTensor(item_vector).to(device)
interaction_tensor = torch.FloatTensor([interaction]).to(device)

#create a toy example for the user
user_tensor = torch.LongTensor(torch.zeros_like(item_tensor)).to(device)
user_tensor[4]=1 #item_id=4 = Comedy "The Father of the Bride"
user_tensor[111] = 1# item_id=111 Documentary "Anne Frank"
    
output = model(user_tensor,item_tensor).to(device)
loss = loss_fn(output, interaction_tensor)
test_losses.append(loss.item())

print(item_id)
print(output)


pred_tensor = torch.tensor(torch.zeros_like(user_tensor.clone().detach())).to(device)
pred_tensor[item_id] = output

TypeError: expected TensorOptions(dtype=long int, device=cpu, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)) (got TensorOptions(dtype=long int, device=cuda:0, layout=Strided, requires_grad=false (default), pinned_memory=false (default), memory_format=(nullopt)))

In [ ]:
#A toy example where user consumed comedy and documentary items and the comedy item was recommended, so we expect that another comedy item will explain this one

#item_id = 96 # Documentary "Heidi Fleiss"
item_id = 3701 # Comedy "Meet the parents"
#item_id = 190 #Horror "Species"
item_vector = items_values.iloc[item_id]
interaction = 1


item_tensor = torch.LongTensor(item_vector).to(device)
interaction_tensor = torch.FloatTensor([interaction]).to(device)

#create a toy example for the user
#user_tensor = torch.LongTensor(torch.zeros_like(item_tensor)).to(device)
user_tensor = torch.zeros_like(item_tensor).to(item_tensor.device).long()
user_tensor[4]=1 #item_id=4 = Comedy "The Father of the Bride"
user_tensor[111] = 1# item_id=111 Documentary "Anne Frank"
user_tensor[190] = 1# item_id=190 Horror "Species"

output = model(user_tensor,item_tensor).to(device)
loss = loss_fn(output, interaction_tensor)
test_losses.append(loss.item())

print(item_id)
print(output)


pred_tensor = torch.tensor(torch.zeros_like(user_tensor.clone().detach()))
pred_tensor[item_id] = output

In [105]:
masked_vector = find_mask_combined(user_tensor.to(torch.float32), item_tensor.to(torch.float32), pred_tensor.to(torch.float32), alpha_parameter=0.5)
#print(masked_vector.sum())
print(masked_vector[4])
print(masked_vector[111])
print(masked_vector[190])
#print(user_tensor.sum())
#print((user_tensor.to(device) - masked_vector))
#print((user_tensor.to(device) - masked_vector)[4])
#print((user_tensor.to(device) - masked_vector)[111])

tensor([0.5158, 0.5146, 0.5509,  ..., 0.4247, 0.6521, 0.5821], device='cuda:0')
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', requires_grad=True)
tensor(0.5662, device='cuda:0')
tensor(0.5375, device='cuda:0')
tensor(0.3209, device='cuda:0')


In [ ]:
#Additional testing

In [168]:
user_vector = pd.DataFrame(np.zeros((1, num_items)))
user_vector.loc[0,4]=1 #item_id=4 = Comedy "The Father of the Bride"
user_vector.loc[0,111] = 1# item_id=111 Documentary "Anne Frank"

,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tensor([0.4444, 0.3076, 0.7410, 0.5060, 0.5405, 0.4144, 0.4562],
       device='cuda:0')
tensor([ 0.7895,  1.0765,  1.0765,  1.0765, -1.3051, -1.2771,  0.7895],
       device='cuda:0', requires_grad=True)
tensor([[1, 0, 0, 0, 1, 0, 1]], device='cuda:0')


tensor([[ 0.8831,  0.7453,  0.2788,  0.5318, -0.0591, -0.7479,  0.8855]],
       device='cuda:0')

In [95]:
#Define combined loss function (according to the long equation that combines min and max parts)
#Find min (two parameters mask and z in order to find  x_masked
class LossModelCombined2(torch.nn.Module):
        def __init__(self,x, y_true,y_pred,alpha_parameter):
            super().__init__()
            self.x = x.to(device)
            self.y_true = y_true.to(device)
            self.y_pred = y_pred.to(device)
            self.mask = torch.nn.Parameter(torch.zeros_like(self.x, dtype=torch.float32, device=device), requires_grad=True) #torch.tensor(self.y_pred), requires_grad=True)    #
            self.z = torch.nn.Parameter(torch.nn.Parameter(torch.zeros_like(self.x, dtype=torch.float32, device=device), requires_grad=True)) #torch.tensor(self.y_pred), requires_grad=True) 
            self.alpha_parameter = alpha_parameter

        def forward(self):
            #x_masked as defined via mask and z: x_m = x o m + z o (1-m)
            x_masked = (self.mask * self.x.to(device) + self.z.to(device)*(torch.ones_like(self.x).to(device)-self.mask)).to(device)
            #x_masked reversed
            x_masked_reversed = (self.x.to(device) * (torch.ones_like(self.x).to(device)-self.mask) + self.z.to(device)*self.mask).to(device)
            #cross entropy loss between x_masked and y_pred that we got from the test item
            cross_entropy_loss = nn.functional.cross_entropy(self.x - x_masked.round(), self.y_pred).to(device)
            #cross entropy on inversed function
            cross_entropy_inversed_loss = nn.functional.cross_entropy(self.x -x_masked_reversed.round(), self.y_pred).to(device)
            #regularization function
            l1_loss = torch.mean(torch.abs(x_masked)).to(device)
            #loss function as defined in the paper (the long one)
            loss = cross_entropy_loss + self.alpha_parameter * l1_loss - self.alpha_parameter * cross_entropy_inversed_loss
            return loss   

In [96]:
#Test items to check the function

In [97]:
x = torch.tensor(np.array([[1,0,0,0,1,0,1]])).to(device)

In [98]:
y_true = torch.tensor(np.array([[0,0,0,0,0,1,0]])).to(device)

In [99]:
#y_pred = torch.tensor(np.array([[0.05,0,0,0,0.3,0.6,0.05]])).to(device)
y_pred = torch.tensor(np.array([[0,0,0,0,0,0.67,0]])).to(device)

In [100]:
#find_mask(x, y_true, y_pred, alpha_parameter=0.5, temperature=1.0, num_samples=10)

In [101]:
find_mask_combined(x, y_true, y_pred, alpha_parameter=0.5, temperature=1.0, num_samples=10)

tensor([0.3277, 0.7611, 0.4077, 0.4864, 0.6112, 0.6554, 0.5555],
       device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0.], device='cuda:0', requires_grad=True)


tensor([[0.3277, 0.0000, 0.0000, 0.0000, 0.6112, 0.0000, 0.5555]],
       device='cuda:0')

In [ ]:
#Evaluation metrics

In [ ]:
def get_hit_ratio(ranked_list, target_item):
    for item in ranked_list:
        if item == target_item:
            return 1
    return 0

In [ ]:
def get_ndcg(ranked_list, target_item):
    for i in range(len(ranked_list)):
        item = ranked_list[i]
        if item == target_item:
            return np.reciprocal(np.log2(i+2))      
    return 0